
# Курсовая работа: Оптимизация расписания маршрутных автобусов

### Введение
Данная работа посвящена разработке алгоритмов для оптимизации расписания маршрутных автобусов. Основной целью является минимизация затрат при соблюдении всех заданных ограничений, таких как рабочее время водителей, перерывы и часы пик.



### Цель работы
Разработать алгоритмы и программное обеспечение для оптимизации расписания маршрутных автобусов.

### Задачи
1. Проанализировать входные данные и ограничения задачи.
2. Реализовать два метода решения: метод "влоб" и генетический алгоритм.
3. Сравнить методы по критериям: оптимальность решения, удобство использования программы, качество документации.
4. Предоставить расписание и выводы в отчёте.


### Определение структуры задачи

In [15]:
# Определяем класс Route (Маршрут)
class Route:
    def __init__(self, name, stops, is_cyclic, time_required):
        self.name = name  # Название маршрута
        self.stops = stops  # Остановки на маршруте
        self.is_cyclic = is_cyclic  # Является ли маршрут цикличным
        self.time_required = time_required  # Время выполнения маршрута

# Определяем класс Driver (Водитель)
class Driver:
    def __init__(self, driver_type, max_hours, breaks, shift_schedule):
        self.driver_type = driver_type  # Тип водителя: 'A' или 'B'
        self.max_hours = max_hours  # Максимальное количество рабочих часов
        self.breaks = breaks  # Перерывы водителя
        self.shift_schedule = shift_schedule  # Расписание смен водителя

# Определяем класс Bus (Автобус)
class Bus:
    def __init__(self, id, route=None, driver=None):
        self.id = id  # Идентификатор автобуса
        self.route = route  # Назначенный маршрут
        self.driver = driver  # Назначенный водитель

# Определяем ScheduleManager для управления маршрутами, автобусами и водителями
class ScheduleManager:
    def __init__(self):
        self.routes = []  # Список маршрутов
        self.buses = []  # Список автобусов
        self.drivers = []  # Список водителей

    def add_route(self, route):
        self.routes.append(route)  # Добавление маршрута в список

    def add_bus(self, bus):
        self.buses.append(bus)  # Добавление автобуса в список

    def add_driver(self, driver):
        self.drivers.append(driver)  # Добавление водителя в список

    def print_schedule(self):
        # Вывод расписания для всех автобусов
        for bus in self.buses:
            print(f"Автобус {bus.id}: Маршрут {bus.route.name if bus.route else 'Нет'}, Водитель {bus.driver.driver_type if bus.driver else 'Нет'}")


In [16]:
if __name__ == "__main__":
    # Инициализация ScheduleManager (Менеджера расписаний)
    manager = ScheduleManager()

    # Добавление маршрутов
    manager.add_route(Route("Маршрут 1", [1, 2, 3, 2, 1], True, 60))  # Цикличный маршрут
    manager.add_route(Route("Маршрут 2", [1, 2, 3, 4, 5, 1], False, 70))  # Конечный маршрут

    # Добавление автобусов
    for i in range(1, 9):  # Создание автобусов с идентификаторами от 1 до 8
        manager.add_bus(Bus(i))

    # Добавление водителей
    manager.add_driver(Driver("A", 80, [4, 1], "06:00-14:00"))  # Увеличено максимальное количество часов
    manager.add_driver(Driver("B", 120, [2, 15], "06:00-06:00"))  # 24-часовая смена

    # Вывод начального расписания
    manager.print_schedule()


Автобус 1: Маршрут Нет, Водитель Нет
Автобус 2: Маршрут Нет, Водитель Нет
Автобус 3: Маршрут Нет, Водитель Нет
Автобус 4: Маршрут Нет, Водитель Нет
Автобус 5: Маршрут Нет, Водитель Нет
Автобус 6: Маршрут Нет, Водитель Нет
Автобус 7: Маршрут Нет, Водитель Нет
Автобус 8: Маршрут Нет, Водитель Нет


### Метод "Влоб" (Brute Force)

Метод "влоб" предполагает полный перебор всех возможных комбинаций автобусов и водителей для составления расписания.  
Этот метод гарантирует нахождение оптимального решения, однако его производительность может быть низкой при большом количестве входных данных.

**Шаги метода:**
1. Перебрать все маршруты.
2. Для каждого маршрута попробовать все возможные комбинации автобусов и водителей.
3. Проверить ограничения (рабочее время, перерывы и часы пик).
4. Составить расписание с минимальными затратами.


In [17]:
from itertools import permutations

# Определить часы пик
PEAK_HOURS = [(7, 9), (17, 19)]

def is_in_peak_hours(start_time):
    """Check if a route start time is in peak hours."""
    hour = int(start_time.split(":")[0])
    for peak_start, peak_end in PEAK_HOURS:
        if peak_start <= hour < peak_end:
            return True
    return False

def brute_force_schedule(manager):
    best_schedule = None
    min_buses_used = float('inf')
    
   # Сгенерировать все перестановки маршрутов
    for route_perm in permutations(manager.routes):
        buses_used = set()
        drivers_used = set()
        current_schedule = []
        start_time = 6  # Начиная с 6:00 утра
        
        for route in route_perm:
            route_assigned = False
            for bus in manager.buses:
                if bus.id not in buses_used:
                    for driver in manager.drivers:
                        if driver.max_hours >= route.time_required and driver.driver_type not in drivers_used:
                            start_hour = f"{start_time}:00"
                            route_type = "Cyclic" if route.is_cyclic else "Terminal"
                            peak_info = " (PEAK HOUR)" if is_in_peak_hours(start_hour) else ""

                            print(f"Route {route.name} ({route_type}) starts at {start_hour}{peak_info}")
                            
                            # Назначить этот автобус и водителя
                            current_schedule.append((bus.id, route.name, driver.driver_type, start_hour, route_type))
                            buses_used.add(bus.id)
                            drivers_used.add(driver.driver_type)
                            route_assigned = True
                            start_time += 1  # Увеличить время начала
                            break
                    if route_assigned:
                        break

            if not route_assigned:
                print(f"Warning: No valid assignment for route {route.name}.")
        
        # Проверить, использует ли это расписание меньше автобусов
        if len(buses_used) < min_buses_used:
            min_buses_used = len(buses_used)
            best_schedule = current_schedule

    return best_schedule


In [18]:
if __name__ == "__main__":
    # # Инициализировать ScheduleManager
    manager = ScheduleManager()

    # Добавить маршруты
    manager.add_route(Route("Route 1", [1, 2, 3, 2, 1], True, 60))
    manager.add_route(Route("Route 2", [1, 2, 3, 4, 5, 1], False, 70))

    # Добавить автобусы
    for i in range(1, 9):
        manager.add_bus(Bus(i))

    # Добавить водителей
    manager.add_driver(Driver("A", 80, [4, 1], "06:00-14:00"))  # Increased max_hours
    manager.add_driver(Driver("B", 120, [2, 15], "06:00-06:00"))
    
    # Распечатать начальное расписание
    manager.print_schedule()
# Сгенерировать расписание методом подбора
schedule = brute_force_schedule(manager)

# Распечатать сгенерированное расписание с указанием времени и типа маршрута
print("\nGenerated Schedule (Brute Force with Route Types and Timings):")
if schedule:
    for entry in schedule:
        print(f"Bus {entry[0]} assigned to {entry[1]} ({entry[4]} Route) with Driver {entry[2]} starting at {entry[3]}")
else:
    print("No valid schedule found.")


Автобус 1: Маршрут Нет, Водитель Нет
Автобус 2: Маршрут Нет, Водитель Нет
Автобус 3: Маршрут Нет, Водитель Нет
Автобус 4: Маршрут Нет, Водитель Нет
Автобус 5: Маршрут Нет, Водитель Нет
Автобус 6: Маршрут Нет, Водитель Нет
Автобус 7: Маршрут Нет, Водитель Нет
Автобус 8: Маршрут Нет, Водитель Нет
Route Route 1 (Cyclic) starts at 6:00
Route Route 2 (Terminal) starts at 7:00 (PEAK HOUR)
Route Route 2 (Terminal) starts at 6:00
Route Route 1 (Cyclic) starts at 7:00 (PEAK HOUR)

Generated Schedule (Brute Force with Route Types and Timings):
Bus 1 assigned to Route 1 (Cyclic Route) with Driver A starting at 6:00
Bus 2 assigned to Route 2 (Terminal Route) with Driver B starting at 7:00


In [19]:
import random

# Функция проверки на пиковые часы
def is_in_peak_hours(start_time):
    """Проверяет, попадает ли время в пиковые часы."""
    hour = int(start_time.split(":")[0])
    for peak_start, peak_end in PEAK_HOURS:
        if peak_start <= hour < peak_end:
            return True
    return False

def genetic_algorithm_schedule(manager, generations=100, population_size=10):
   
    # Функция для создания случайного расписания
    def create_random_schedule():
        schedule = []
        buses_used = set()
        drivers_used = set()
        start_time = 6  # Начало расписания в 6:00
        for route in manager.routes:
            available_buses = [bus for bus in manager.buses if bus.id not in buses_used]
            available_drivers = [driver for driver in manager.drivers if driver.driver_type not in drivers_used and driver.max_hours >= route.time_required]
            
            if not available_buses or not available_drivers:
                print(f"Warning: No resources available for route {route.name}.")
                continue

            bus = random.choice(available_buses)
            driver = random.choice(available_drivers)

            # Назначить автобус и водителя
            route_type = "Cyclic" if route.is_cyclic else "Terminal"
            start_hour = f"{start_time}:00"
            peak_info = " (PEAK HOUR)" if is_in_peak_hours(start_hour) else ""

            print(f"Route {route.name} ({route_type}) starts at {start_hour}{peak_info}")
            
            schedule.append((bus.id, route.name, driver.driver_type, start_hour, route_type))
            buses_used.add(bus.id)  # Mark bus as used
            drivers_used.add(driver.driver_type)  # Mark driver as used
            start_time += 1
        return schedule


    # Функция оценки приспособленности
    def fitness(schedule):
        bus_penalty = len(schedule) - len(set(entry[0] for entry in schedule))  # Penalty for reused buses
        driver_penalty = len(schedule) - len(set(entry[2] for entry in schedule))  # Penalty for reused drivers
        return len(set(entry[0] for entry in schedule)) + bus_penalty + driver_penalty

    # Функция скрещивания (crossover)
    def crossover(parent1, parent2):
        split_point = len(parent1) // 2
        return parent1[:split_point] + parent2[split_point:]

    # Функция мутации (mutation)
    def mutate(schedule):
        index = random.randint(0, len(schedule) - 1)
        used_buses = {entry[0] for entry in schedule if entry != schedule[index]}
        used_drivers = {entry[2] for entry in schedule if entry != schedule[index]}

        available_buses = [bus for bus in manager.buses if bus.id not in used_buses]
        available_drivers = [driver for driver in manager.drivers if driver.driver_type not in used_drivers]

        if available_buses and available_drivers:
            bus = random.choice(available_buses)
            driver = random.choice(available_drivers)
            schedule[index] = (bus.id, schedule[index][1], driver.driver_type, schedule[index][3], schedule[index][4])

    # Генерация начальной популяции
    population = [create_random_schedule() for _ in range(population_size)]

    # Эволюция популяции
    for generation in range(generations):
        population = sorted(population, key=fitness)
        population = population[:population_size // 2]  # Отбор лучших решений
        population = random.choices(population, weights=[1/fitness(s) for s in population], k=population_size)

        next_generation = []
        while len(next_generation) < population_size:
            parent1, parent2 = random.sample(population, 2)
            child = crossover(parent1, parent2)
            if random.random() < 0.3:  # Повышенная вероятность мутации
                mutate(child)
            next_generation.append(child)

        population = next_generation

    # Возврат лучшего расписания
    best_schedule = min(population, key=fitness)
    return best_schedule


In [20]:
if __name__ == "__main__":
    # Инициализация ScheduleManager (Менеджера расписаний)
    manager = ScheduleManager()

    # Добавление маршрутов
    manager.add_route(Route("Маршрут 1", [1, 2, 3, 2, 1], True, 60))  # Цикличный маршрут
    manager.add_route(Route("Маршрут 2", [1, 2, 3, 4, 5, 1], False, 70))  # Конечный маршрут

    # Добавление автобусов
    for i in range(1, 9):  # Создание автобусов с идентификаторами от 1 до 8
        manager.add_bus(Bus(i))

    # Добавление водителей
    manager.add_driver(Driver("A", 80, [4, 1], "06:00-14:00"))  # Увеличено максимальное количество часов
    manager.add_driver(Driver("B", 120, [2, 15], "06:00-06:00"))  # 24-часовая смена

    # Вывод начального расписания
    manager.print_schedule()

# Генерация расписания с использованием генетического алгоритма
ga_schedule = genetic_algorithm_schedule(manager)

# Вывод сгенерированного расписания
print("\nСгенерированное расписание (Генетический алгоритм с типами маршрутов и временем):")
if ga_schedule:
    for entry in ga_schedule:
        print(f"Автобус {entry[0]} назначен на {entry[1]} ({entry[4]} маршрут) с водителем {entry[2]} начиная с {entry[3]}")
else:
    print("Не удалось найти подходящее расписание.")


Автобус 1: Маршрут Нет, Водитель Нет
Автобус 2: Маршрут Нет, Водитель Нет
Автобус 3: Маршрут Нет, Водитель Нет
Автобус 4: Маршрут Нет, Водитель Нет
Автобус 5: Маршрут Нет, Водитель Нет
Автобус 6: Маршрут Нет, Водитель Нет
Автобус 7: Маршрут Нет, Водитель Нет
Автобус 8: Маршрут Нет, Водитель Нет
Route Маршрут 1 (Cyclic) starts at 6:00
Route Маршрут 2 (Terminal) starts at 7:00 (PEAK HOUR)
Route Маршрут 1 (Cyclic) starts at 6:00
Route Маршрут 2 (Terminal) starts at 7:00 (PEAK HOUR)
Route Маршрут 1 (Cyclic) starts at 6:00
Route Маршрут 2 (Terminal) starts at 7:00 (PEAK HOUR)
Route Маршрут 1 (Cyclic) starts at 6:00
Route Маршрут 2 (Terminal) starts at 7:00 (PEAK HOUR)
Route Маршрут 1 (Cyclic) starts at 6:00
Route Маршрут 2 (Terminal) starts at 7:00 (PEAK HOUR)
Route Маршрут 1 (Cyclic) starts at 6:00
Route Маршрут 2 (Terminal) starts at 7:00 (PEAK HOUR)
Route Маршрут 1 (Cyclic) starts at 6:00
Route Маршрут 2 (Terminal) starts at 7:00 (PEAK HOUR)
Route Маршрут 1 (Cyclic) starts at 6:00
Route 

### Пример ввода данных

In [21]:
if __name__ == "__main__":
    # Инициализация ScheduleManager (Менеджера расписаний)
    manager = ScheduleManager()

    # Добавление маршрутов
    manager.add_route(Route("Маршрут 1", [1, 2, 3, 2, 1], True, 60))  # Цикличный маршрут
    manager.add_route(Route("Маршрут 2", [1, 2, 3, 4, 5, 1], False, 70))  # Конечный маршрут

    # Добавление автобусов
    for i in range(1, 9):  # Создание автобусов с идентификаторами от 1 до 8
        manager.add_bus(Bus(i))

    # Добавление водителей
    manager.add_driver(Driver("A", 80, [4, 1], "06:00-14:00"))  # Увеличено максимальное количество часов
    manager.add_driver(Driver("B", 120, [2, 15], "06:00-06:00"))  # 24-часовая смена

    # Вывод начального расписания
    manager.print_schedule()

    # Генерация расписания с использованием метода полного перебора
    schedule = brute_force_schedule(manager)

    # Вывод сгенерированного расписания (метод полного перебора)
    print("\nСгенерированное расписание (Полный перебор):")
    if schedule:
        for entry in schedule:
            print(f"Автобус {entry[0]} назначен на {entry[1]} с водителем {entry[2]}")
    else:
        print("Не удалось найти подходящее расписание.")

    # Генерация расписания с использованием генетического алгоритма
    ga_schedule = genetic_algorithm_schedule(manager)

    # Вывод сгенерированного расписания (генетический алгоритм)
    print("\nСгенерированное расписание (Генетический алгоритм):")
    if ga_schedule:
        for entry in ga_schedule:
            print(f"Автобус {entry[0]} назначен на {entry[1]} с водителем {entry[2]}")
    else:
        print("Не удалось найти подходящее расписание.")


Автобус 1: Маршрут Нет, Водитель Нет
Автобус 2: Маршрут Нет, Водитель Нет
Автобус 3: Маршрут Нет, Водитель Нет
Автобус 4: Маршрут Нет, Водитель Нет
Автобус 5: Маршрут Нет, Водитель Нет
Автобус 6: Маршрут Нет, Водитель Нет
Автобус 7: Маршрут Нет, Водитель Нет
Автобус 8: Маршрут Нет, Водитель Нет
Route Маршрут 1 (Cyclic) starts at 6:00
Route Маршрут 2 (Terminal) starts at 7:00 (PEAK HOUR)
Route Маршрут 2 (Terminal) starts at 6:00
Route Маршрут 1 (Cyclic) starts at 7:00 (PEAK HOUR)

Сгенерированное расписание (Полный перебор):
Автобус 1 назначен на Маршрут 1 с водителем A
Автобус 2 назначен на Маршрут 2 с водителем B
Route Маршрут 1 (Cyclic) starts at 6:00
Route Маршрут 2 (Terminal) starts at 7:00 (PEAK HOUR)
Route Маршрут 1 (Cyclic) starts at 6:00
Route Маршрут 2 (Terminal) starts at 7:00 (PEAK HOUR)
Route Маршрут 1 (Cyclic) starts at 6:00
Route Маршрут 2 (Terminal) starts at 7:00 (PEAK HOUR)
Route Маршрут 1 (Cyclic) starts at 6:00
Route Маршрут 2 (Terminal) starts at 7:00 (PEAK HOUR)
Ro